# L2: Evaluate Inputs: Classification

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

### Classify customer queries to handle different cases
- It's important that you will set delimiter so the model knows what's the question.
- "####" is a good delimiter as it's act as 1 token.
- Primary = high level choice.
- Secondary = detaisl of high level choice.
- Idea: we define the high and low level choices. Then the model will acts as classifier.
- Output into a JSON will allow you to read the output as an object.

In [3]:
delimiter = "####"
system_message = f"""
You will be provided with customer service queries. \
The customer service query will be delimited with \
{delimiter} characters.
Classify each query into a primary category \
and a secondary category. 
Provide your output in json format with the \
keys: primary and secondary.

Primary categories: Billing, Technical Support, \
Account Management, or General Inquiry.

Billing secondary categories:
Unsubscribe or upgrade
Add a payment method
Explanation for charge
Dispute a charge

Technical Support secondary categories:
General troubleshooting
Device compatibility
Software updates

Account Management secondary categories:
Password reset
Update personal information
Close account
Account security

General Inquiry secondary categories:
Product information
Pricing
Feedback
Speak to a human

"""
user_message = f"""\
I want you to delete my profile and all of my user data"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

{
    "primary": "Account Management",
    "secondary": "Close account"
}


In [4]:
user_message = f"""\
Tell me more about your flat screen tvs"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

{
  "primary": "General Inquiry",
  "secondary": "Product information"
}  



### Create an example about the questions for the new hire support
- Interns will ask about training, contacts, event, team.

In [5]:
delimiter = "####"

system_message = f"""
You will be a software engineer manager at BlackRock who helps new hires.\
The inquiry query will be delimited with {delimiter} characters.

Classify each query into a primary category and a secondary category.

Provide your output in json format with the key: primary and secondary.

Primary categories: Trainings, Contacts, Events, or Teams.

Trainings secondary categories:
Software engineer training
Machine learning training
Finance training
All training

Contacts secondary categories:
Human resource group
IT group

Events secondary categories:
Analyst connect events
Volunteer events
Sport events

Team categories
Manager
Senior engineer
Fellow engineer
"""

In [6]:
user_message = f"""
I want you to give me all the trainings.
"""

messages = [
    {'role': 'system',
     'content': system_message},
    {'role': 'user',
     'content': f"{delimiter}{user_message}{delimiter}"},
]

response = get_completion_from_messages(messages)
print(response)

{
  "primary": "Trainings",
  "secondary": "All training"
}
